# This is  a research notebook to try to find correlation between hyperwave and volume 

In [1]:
!pip install --upgrade pip
!pip install quandl

Requirement already up-to-date: pip in /home/nbuser/anaconda3_501/lib/python3.6/site-packages (18.0)
    100% |████████████████████████████████| 51kB 3.9MB/s ta 0:00:01
  Running setup.py bdist_wheel for inflection ... done
  Stored in directory: /home/nbuser/.cache/pip/wheels/9f/5a/d3/6fc3bf6516d2a3eb7e18f9f28b472110b59325f3f258fe9211
Successfully built inflection


In [2]:
import time
from datetime import datetime, date, time, timedelta
import json
import requests
import os.path as path

from scipy.spatial import ConvexHull

import quandl
quandl.ApiConfig.api_key = "cEofBzyzyihN3fj62kp4"

import pandas as pd
import numpy as np

import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.tools.set_credentials_file(username='davzucky', api_key='aZw7LRJOSDcPJyIk2G0U')
# This is to avoid warning when dividing by zero
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'warn', 'invalid': 'warn', 'over': 'warn', 'under': 'ignore'}

In [3]:
path.abspath('.')

'/home/nbuser/library'

In [4]:
# mydata =  pd.DataFrame(quandl.get("FRED/GDP",returns="numpy", collapse="weekly",qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high'] },))
# mydata['diff'] = mydata['open'] - mydata['close']


In [5]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(mydata)

In [6]:
# f = mydata
# # display(f['open'])
# f = f.set_index('date')

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(weekly_data )

## This is the place where you setup the symbol and phases

In [7]:
data_3DSystemsCorp ={
    'name':'3D SYSTEMS CORP',
    'data_source':'quandl',
    'symbol':'DDD',
}


data_Valeant = {
    'name':'Baush health Company. Ex Valeant',
    'data_source':'quandl',
    'symbol':'VRX',
}

data_Amazon = {
    'name':'Amazon',
    'data_source':'investopedia',
    'symbol':'AMZN',
}

data_Square = {
    'name':'Square',
    'data_source':'investopedia',
    'symbol':'SQ',
}

data_Netflix= {
    'name':'Netflix',
    'data_source':'investopedia',
    'symbol':'NFLX',
}

data_Apple = {
    'name':'Apple',
    'data_source':'investopedia',
    'symbol':'AAPL',
}

data_Intuit = {
    'name':'Intuit',
    'data_source':'investopedia',
    'symbol':'INTU',
}

data_Gilead_Sciences = {
    'name':'Gilead Sciences',
    'data_source':'investopedia',
    'symbol':'GILD',
}

data_Abiomed = {
    'name':'Abiomed',
    'data_source':'investopedia',
    'symbol':'ABMD',
}

data_Alphabet = {
    'name':'Alphabet',
    'data_source':'investopedia',
    'symbol':'GOOGL',
}

data_BTCUSD = {
    'name':'BTCUSD',
    'data_source':'CryptoCompare',
    'symbol':'BTC-USD',
}

data_DowJones = {
    'name':'Dow Jones',
    'data_source':'investopedia',
    'symbol':'^DJI',
}

data_Chevron = {
    'name':'Chevron',
    'data_source':'investopedia',
    'symbol':'CVX',
}


data_DowJones_1920_1933 = {
    'name':'Dow Jones 1920-1933',
    'data_source':'LocalData',
    'symbol':'DowJones_1920_1933',
}

data_DowJones = {
    'name':'Dow Jones Industrial',
    'data_source':'LocalData',
    'symbol':'DowJones_Full',
}


data_10Y_Treasury_Note = {
    'name':'10y_usd_treasury_note',
    'data_source':'LocalData',
    'symbol':'10y_usd_treasury_note',
}


data_Caterpilar = {
    'name':'Caterpilar',
    'data_source':'investopedia',
    'symbol':'CAT',
}
data_DR_Horton = {
    'name':'D.R. Horton',
    'data_source':'investopedia',
    'symbol':'DHI',
}


In [8]:
company_setup = data_DR_Horton  
name = company_setup ['name'] 
data_source = company_setup ['data_source'] 
symbol = company_setup ['symbol']

# constant used for other computation
root_date = datetime(1800, 1, 6)

# Helper function 
This section contain helper function that are here to load and clean the raw data

In [9]:
def get_nb_weeks(row, base_date):
    return int((row["date"]-base_date).days/7)

def add_weekid_and_price_is_closing_up(df, base_date ):
    df['is_price_closing_up'] = df.close > df.close.shift()
    df['weekId'] = df.apply (lambda row: get_nb_weeks (row, base_date),axis=1)
    if "volume" in list(df.columns.values):
        df = df.drop("volume", axis=1)
    return df.sort_values(by="date")

class Investopedia_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe
    
    def _clean_data(self, df):
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('Date')])
        df = df.rename(columns={'Adj. Close':'close', 'Low':'low', 'Open':'open', 'High':'high','Volume':'volume'})
        df = df.set_index('date')
        df['date'] = df.index
        return df

    def _fetch_data(self): 
        url_symbol = "https://www.investopedia.com/markets/api/partial/historical/?Symbol={}&Type=Historical+Prices&Timeframe={}&StartDate=Jan+01%2C+1900".format(self._symbol, self._timeframe)
        df_list = pd.read_html(url_symbol, header=0, parse_dates=True)
        df_price = df_list[0].dropna()
        return df_price
    
    def get_dataframe(self):
        raw_data = self._fetch_data()
        return self._clean_data(raw_data)

class Quandl_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-FRI').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        daily_tickers = quandl.get_table('WIKI/PRICES', \
                                         ticker = [self._symbol], \
                                         qopts = { 'columns': ['ticker', 'date', 'close', 'open', 'low', 'high', 'volume'] }, \
                                         date = { 'gte': '1900-01-01'}, \
                                         paginate=True) #, 'lte': '2016-12-31' 
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class CryptoCompare_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._symbol = symbol
        self._timeframe = timeframe

    def _aggregate_ticker_weekly(self, df):    
        open = df.open.resample('W-MON').last()
        close = df.close.resample('W-SUN').last().resample('W-MON').last()
        high = df.high.resample('W-MON').max()
        low = df.low.resample('W-MON').min()
        vol = df.volume.resample('W-MON').sum() 
        
        weekly_data = pd.concat([open, close, high, low, vol], axis=1)
        weekly_data ['date'] = weekly_data .index
        return weekly_data

    def _fetch_daily_data(self):
        from_symbol, to_symbol  = self._symbol.split('-')
        url = "https://min-api.cryptocompare.com/data/histoday?fsym={}&tsym={}&allData=true&aggregate=3&e=CCCAGG".format( \
                                                                                                                        from_symbol, \
                                                                                                                        to_symbol)

        r = requests.get(url)
        array = json.dumps(r.json())
        
        data = json.loads(array)
        daily_tickers = pd.DataFrame(data["Data"])
        daily_tickers['date'] = pd.to_datetime(daily_tickers['time'],unit='s')
        daily_tickers = daily_tickers.rename( columns={"volumeto": "volume"})
        daily_tickers = daily_tickers.set_index('date')
        daily_tickers['date'] = daily_tickers.index
        return daily_tickers
    
    def get_dataframe(self):
        daily_dataframe = self._fetch_daily_data()
        if( self._timeframe == 'daily'):
            return daily_dataframe.dropna()
        
        return self._aggregate_ticker_weekly(daily_dataframe).dropna()

class LocalData_Loader():
    def __init__(self, symbol, timeframe='weekly'):
        self._file_name = "{}.csv".format(symbol)
        self._timeframe = timeframe

    
    def get_dataframe(self):
        file_path = path.join(path.abspath('.'), 'data', self._file_name)
        df = pd.read_csv(file_path, header=0, parse_dates=True)
        df = df.rename(columns={column: column.lower() for column in df.columns})
        df.loc[:,('date')] = pd.to_datetime(df.loc[:,('date')])
        df = df.set_index('date')
        df['date'] = df.index
        return df
        

# https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG
    
sources = {
    "investopedia": Investopedia_Loader,
    "quandl": Quandl_Loader,
    "CryptoCompare": CryptoCompare_Loader,
    "LocalData": LocalData_Loader
}


def get_historical_data(symbol, source, base_date, timeframe='weekly'):
    source_class = sources[source](symbol, timeframe)
    df_raw = source_class.get_dataframe()
    df_with_weekId = add_weekid_and_price_is_closing_up( df_raw, base_date )
    df_with_weekId = df_with_weekId.reset_index(drop=True)
    df_with_weekId = df_with_weekId.set_index('weekId')
    df_with_weekId['weekId'] = df_with_weekId.index
    return df_with_weekId

In [10]:
# url = "https://min-api.cryptocompare.com/data/histoday?fsym=BTC&tsym=USD&allData=true&aggregate=3&e=CCCAGG"
# df = pd.read_json(url)
# df = pd.DataFrame(df.locations.values.tolist())['Data']
# display(df)

# import json
# import pandas as pd
# import requests
# r = requests.get(url)
# # print r.json() #
# array = json.dumps(r.json())
# # print(data["Data"])
# data = json.loads(array)
# df = pd.DataFrame(data["Data"])
# df['date'] = pd.to_datetime(df['time'],unit='s')
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df)

## Function that allow to calculate the weekid from a start date

In [11]:


def get_weekId_max_price(df):
    return df.loc[ df.loc[:,('close')].argmax()]['weekId']

def get_weekId_min_price(df):
    return df.loc[ df.loc[:,('close')].argmin()]['weekId']


In [12]:
# def get_weekId_cartesian_product(df):
#     df_from = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_from", "close": "close_from"})
#     df_to = df.loc[:,('weekId',"close")].rename(index=str, columns={"weekId": "weekId_to", "close": "close_to"})
#     df_cartesian = df_from.assign(foo=1).merge(df_to.assign(foo=1)).drop('foo', 1)

#     return df_cartesian [(df_cartesian .weekId_from < df_cartesian .weekId_to)]    

# From here we start fetching the data
Above was only about setting up some basic function

In [13]:
# df_daily_price_raw = get_raw_historical_data(symbol, 'Daily')
df_weekly_price  = get_historical_data(symbol, data_source, root_date, 'Weekly')

# df_weekly_price = add_weekid_and_price_is_closing_up(df_weekly_price_raw)

max_price_weekId = get_weekId_max_price(df_weekly_price)

df_weekly_price_until_max = df_weekly_price [(df_weekly_price.weekId <= max_price_weekId)]
min_price_before_max_weekId = get_weekId_min_price(df_weekly_price_until_max)
df_weekly_price_until_max = df_weekly_price_until_max [(df_weekly_price_until_max.weekId >= min_price_before_max_weekId)]
# weekId_close_from_to = get_weekId_cartesian_product(df_weekly_price_until_max)

In [14]:
print(max_price_weekId)
print(min_price_before_max_weekId)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_weekly_price_until_max)

11374
10260


,Date,open,high,low,close,date,is_price_closing_up,weekId
weekId,,,,,,,,
10260,"Aug 30, 1996",1.72,1.75,1.69,1.75,1996-08-30,False,10260
10261,"Sep 06, 1996",1.93,1.96,1.91,1.96,1996-09-06,False,10261
10262,"Sep 13, 1996",1.96,2.01,1.96,1.99,1996-09-13,False,10262
10263,"Sep 20, 1996",2.01,2.09,2.01,2.09,1996-09-20,True,10263
10264,"Sep 27, 1996",2.04,2.06,2.04,2.06,1996-09-27,False,10264
10265,"Oct 04, 1996",2.01,2.06,2.01,2.06,1996-10-04,True,10265
10266,"Oct 11, 1996",1.96,1.99,1.96,1.99,1996-10-11,False,10266
10267,"Oct 18, 1996",1.96,1.99,1.96,1.99,1996-10-18,True,10267
10268,"Oct 25, 1996",2.01,2.01,1.91,1.96,1996-10-25,True,10268


In [15]:
# print(max_price_weekId)
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price)
# df_weekly_price.ix[:df_weekly_price_until_max]

In [16]:
def get_line_slope_and_origine(df, x1_col_name = 'x1', \
                               y1_col_name = 'y1', \
                               x2_col_name = 'x2', \
                               y2_col_name = 'y2', \
                               m_col_name = 'm', \
                               b_col_name = 'b'):
    df[m_col_name] = (df[y1_col_name] - df[y2_col_name]) / (df[x1_col_name] - df[x2_col_name])
    df[b_col_name] = df[y1_col_name] - (df[x1_col_name] * df[m_col_name])
    return df

def get_normalize_column(df, column_name = 'close'):
    column_name_normalize = '{}_normalize'.format(column_name )
    max_value = df.loc[df[column_name ].idxmax()][column_name ]
    min_value = df.loc[df[column_name ].idxmin()][column_name ]

    df[column_name_normalize] = (df.loc[:,(column_name)] - min_value)/ (max_value - min_value ) 
    return df

def get_mean_square_error(y_true, y_pred):
#     y_square_diff = np.square(y_true-y_pred)
    y_square_diff = y_true - y_pred
    return np.sum(y_square_diff) / len(y_true)

def nb_is_lower(y_true, y_pred):
    lower_item = y_true[y_true <= y_pred]
    return len(lower_item )
    
def get_y(x, m, b):
    return x * m + b

def calculate_mean_square_error(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return get_mean_square_error(df['close'], y_pred)

def nb_cut_price_low(row, df):
    y_pred = get_y(df['weekId'], row['m'], row['b'])
    return nb_is_lower(df['low'], y_pred)

# def normalize_column(df, column_name = 'close'):
    

def get_hull_tangent(df_input, x_column_name = 'weekId', y_column_name = 'close'):
    df_input = get_normalize_column(df_input, 'close')
    df_input= get_normalize_column(df_input, 'weekId')
    hull = ConvexHull(df_input[[x_column_name,  y_column_name]].dropna())
    
    hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
    data_from_to = [{"x1": df_input['weekId'].iloc[pair[0]], \
        "x1_date": root_date + timedelta(weeks = df_input['weekId'].iloc[pair[0]].item()), \
        "x1_normalize": df_input['weekId_normalize'].iloc[pair[0]], \
        "y1": df_input['close'].iloc[pair[0]], \
        "y1_normalize": df_input['close_normalize'].iloc[pair[0]], \
        "x2": df_input['weekId'].iloc[pair[1]], \
        "x2_date": root_date + timedelta(weeks = df_input['weekId'].iloc[pair[1]].item()), \
        "x2_normalize": df_input['weekId_normalize'].iloc[pair[1]], \
        "y2": df_input['close'].iloc[pair[1]], \
        "y2_normalize": df_input['close_normalize'].iloc[pair[1]]} for pair in hull_results]
    df = pd.DataFrame(data_from_to)    
    df = get_line_slope_and_origine(df)
    df = get_line_slope_and_origine(df, \
                                x1_col_name = 'x1_normalize', \
                                y1_col_name = 'y1_normalize', \
                                x2_col_name = 'x2_normalize', \
                                y2_col_name = 'y2_normalize', \
                                m_col_name = 'm_normalize', \
                                b_col_name = 'b_normalize')
    df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
    df['angle_normalize'] = np.rad2deg(np.arctan2(df['m_normalize'], 1))
    df['weeks'] = np.abs(df['x1'] - df['x2'])
    df['mean_error'] = df.apply(lambda row: calculate_mean_square_error (row, df_input),axis=1)
    df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df_input),axis=1)
    df['ratio_error_cut'] = df['mean_error'] / df['nb_is_lower']
    df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m_normalize']
    df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m_normalize']
    return df


# def calculate_line_standard(df):
#     df = get_line_slope_and_origine(df)
#     df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
#     df['days'] = np.abs(df['x1'] - df['x2'])
#     df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df),axis=1)
#     df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df),axis=1)
#     df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
#     df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m']
#     df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m']
#     return df

# def calculate_line_normalize(df):
#     df = get_line_slope_and_origine(df, \
#                                     x1_col_name = 'x1_normalize', \
#                                     y1_col_name = 'y1_normalize', \
#                                     x2_col_name = 'x2_normalize', \
#                                     y2_col_name = 'y2_normalize')
    
#     df['angle'] = np.rad2deg(np.arctan2(df['m'], 1))
#     df['days'] = np.abs(df['x1'] - df['x2'])
#     df['mean_square_error'] = df.apply(lambda row: calculate_mean_square_error (row, df),axis=1)
#     df['nb_is_lower'] = df.apply(lambda row: nb_cut_price_low(row, df),axis=1)
#     df['ratio_error_cut'] = df['mean_square_error'] / df['nb_is_lower']
#     df['ratio_slope_y1_normalize'] = df['y1_normalize']/df['m']
#     df['ratio_slope_y2_normalize'] = df['y2_normalize']/df['m']
#     return df


# def get_hull_tangent_standard(df):
#     df = get_normalize_column(df, 'close')
#     df = get_normalize_column(df, 'weekId')
#     return calculate_line_standard(get_hull_tangent(df))


# def get_hull_tangent_normalize(df):
#     df = get_normalize_column(df, 'close')
#     df = get_normalize_column(df, 'weekId')
#     return calculate_line_normalize(get_hull_tangent(df, 'weekId_normalize', 'close_normalize'))
    
    
def get_upper_path(df):
    row = df.iloc[0]
    upper_path = [row['x1'], row['x2']]

    while(row['x2'] != max_price_weekId):
        row = df.loc[df['x1'] == row['x2']].head().iloc[0]
        upper_path.append(row['x2'])

    return upper_path

def delete_path(df, path):
    df_cleaned = df.copy(True)
    
    for x1, x2 in zip(path[:-1], path[1:]):
        df_cleaned = df_cleaned[(np.logical_not((df_cleaned.x1 == x1) & (df_cleaned.x2 == x2))) ]

    return df_cleaned.sort_values(['x1','y2'], ascending=[True, False])

def delete_upper_path(df):
    return delete_path(df, get_upper_path(df.sort_values(['x1','x2'], ascending=[True, False])))

def delete_shorter_path(df):
    df = df.sort_values(['x1','x2'], ascending=[True, False])
    path1 = get_upper_path(df)
    path2 = get_upper_path(df.iloc[1:])
    path = path1 if len(path1) < len(path2) else path2
    return delete_path(df, path)
    

In [17]:
# hull_results = [ [min(pair[0], pair[1]), max(pair[0], pair[1])] for pair in hull.simplices]
# data_from_to = [{"x1": df_weekly_price_until_max['weekId'].iloc[pair[0]], \
#    "y1": df_weekly_price_until_max['close'].iloc[pair[0]], \
#    "x2": df_weekly_price_until_max['weekId'].iloc[pair[1]], \
#    "y2": df_weekly_price_until_max['close'].iloc[pair[1]]} for pair in hull_results]
# df = pd.DataFrame(data_from_to)

In [18]:
def graph_hyperwave(df_data, df_hyperwave):

    x = df_data['weekId'].values
    y =  df_data['close_normalize'].values
    trace0 = go.Scatter(
        x = x,
        y = y,
        mode = 'lines',
        name = 'lines'
    )

    layout = {
    #     'xaxis': {
    #         'range': [0, 7]
    #     },
    #     'yaxis': {
    #         'range': [0, 2.5]
    #     },
        'shapes': [{
                'type': 'line',
                'x0': row[1]['x1'],
                'y0': row[1]['y1'],
                'x1': row[1]['x2'],
                'y1': row[1]['y2'],
                'line': {
                    'color': 'rgb(55, 128, 191)',
                    'width': 3,
                }
            } for row in df_hyperwave.iterrows()
            # Line Vertical
            ]
    }
    data = [trace0]

    # py.iplot(data, filename='line-mode')
    fig = {
        'data': data,
        'layout': layout,
    }

    return fig
#     py.iplot(fig, filename='shapes-lines')

In [19]:
df_hull_tangent = get_hull_tangent(df_weekly_price_until_max)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False]))
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hull_tangent), filename='hyperwave')
# df_hull_tangent = delete_upper_path(df_hull_tangent)

,x1,x1_date,x1_normalize,x2,x2_date,x2_normalize,y1,y1_normalize,y2,y2_normalize,m,b,m_normalize,b_normalize,angle,angle_normalize,weeks,mean_error,nb_is_lower,ratio_error_cut,ratio_slope_y1_normalize,ratio_slope_y2_normalize
1,10260,1996-08-26,0.000000,10898,2008-11-17,0.572711,1.75,0.000000,3.87,0.042147,0.003323,-32.342790,0.073592,0.000000,0.190387,4.208941,638,11.442571,3,3.814190,0.000000,0.572711
5,10260,1996-08-26,0.000000,10261,1996-09-02,0.000898,1.75,0.000000,1.96,0.004175,0.210000,-2152.850000,4.650895,0.000000,11.859779,77.865441,1,-103.676583,1115,-0.092983,0.000000,0.000898
6,10261,1996-09-02,0.000898,10724,2005-07-18,0.416517,1.96,0.004175,34.59,0.652883,0.070475,-721.185637,1.560822,0.002774,4.031264,57.352793,463,-26.100773,1115,-0.023409,0.002675,0.418294
0,10724,2005-07-18,0.416517,11374,2018-01-01,1.000000,34.59,0.652883,52.05,1.000000,0.026862,-253.473138,0.594906,0.405094,1.538683,30.748653,650,-22.044706,1115,-0.019771,1.097456,1.680939
2,10898,2008-11-17,0.572711,11047,2011-09-26,0.706463,3.87,0.042147,8.43,0.132803,0.030604,-329.652685,0.677791,-0.346031,1.752934,34.129065,149,13.652343,4,3.413086,0.062183,0.195935
3,11047,2011-09-26,0.706463,11275,2016-02-08,0.911131,8.43,0.132803,23.52,0.432803,0.066184,-722.706974,1.465789,-0.902723,3.786554,55.697167,228,21.835785,4,5.458946,0.090602,0.295270
4,11275,2016-02-08,0.911131,11321,2016-12-26,0.952424,23.52,0.432803,26.78,0.497614,0.070870,-775.534348,1.569557,-0.997269,4.053749,57.497866,46,23.981678,2,11.990839,0.275749,0.317041
9,11321,2016-12-26,0.952424,11357,2017-09-04,0.984740,26.78,0.497614,35.85,0.677932,0.251944,-2825.483056,5.579843,-4.816761,14.141050,79.839514,36,115.243417,7,16.463345,0.089181,0.121497
10,11357,2017-09-04,0.984740,11359,2017-09-18,0.986535,35.85,0.677932,36.50,0.690855,0.325000,-3655.175000,7.197813,-6.410040,18.004162,82.090465,2,154.693417,3,51.564472,0.094186,0.095981
8,11359,2017-09-18,0.986535,11373,2017-12-25,0.999102,36.50,0.690855,50.64,0.971968,1.010000,-11436.090000,22.368588,-21.376541,45.285051,87.440265,14,525.963417,3,175.321139,0.030885,0.043452


In [20]:
df_hull_tangent.sort_values(['x1','x2'], ascending=[True, False])

,x1,x1_date,x1_normalize,x2,x2_date,x2_normalize,y1,y1_normalize,y2,y2_normalize,...,m_normalize,b_normalize,angle,angle_normalize,weeks,mean_error,nb_is_lower,ratio_error_cut,ratio_slope_y1_normalize,ratio_slope_y2_normalize
1,10260,1996-08-26,0.000000,10898,2008-11-17,0.572711,1.75,0.000000,3.87,0.042147,...,0.073592,0.000000,0.190387,4.208941,638,11.442571,3,3.814190,0.000000,0.572711
5,10260,1996-08-26,0.000000,10261,1996-09-02,0.000898,1.75,0.000000,1.96,0.004175,...,4.650895,0.000000,11.859779,77.865441,1,-103.676583,1115,-0.092983,0.000000,0.000898
6,10261,1996-09-02,0.000898,10724,2005-07-18,0.416517,1.96,0.004175,34.59,0.652883,...,1.560822,0.002774,4.031264,57.352793,463,-26.100773,1115,-0.023409,0.002675,0.418294
0,10724,2005-07-18,0.416517,11374,2018-01-01,1.000000,34.59,0.652883,52.05,1.000000,...,0.594906,0.405094,1.538683,30.748653,650,-22.044706,1115,-0.019771,1.097456,1.680939
2,10898,2008-11-17,0.572711,11047,2011-09-26,0.706463,3.87,0.042147,8.43,0.132803,...,0.677791,-0.346031,1.752934,34.129065,149,13.652343,4,3.413086,0.062183,0.195935
3,11047,2011-09-26,0.706463,11275,2016-02-08,0.911131,8.43,0.132803,23.52,0.432803,...,1.465789,-0.902723,3.786554,55.697167,228,21.835785,4,5.458946,0.090602,0.295270
4,11275,2016-02-08,0.911131,11321,2016-12-26,0.952424,23.52,0.432803,26.78,0.497614,...,1.569557,-0.997269,4.053749,57.497866,46,23.981678,2,11.990839,0.275749,0.317041
9,11321,2016-12-26,0.952424,11357,2017-09-04,0.984740,26.78,0.497614,35.85,0.677932,...,5.579843,-4.816761,14.141050,79.839514,36,115.243417,7,16.463345,0.089181,0.121497
10,11357,2017-09-04,0.984740,11359,2017-09-18,0.986535,35.85,0.677932,36.50,0.690855,...,7.197813,-6.410040,18.004162,82.090465,2,154.693417,3,51.564472,0.094186,0.095981
8,11359,2017-09-18,0.986535,11373,2017-12-25,0.999102,36.50,0.690855,50.64,0.971968,...,22.368588,-21.376541,45.285051,87.440265,14,525.963417,3,175.321139,0.030885,0.043452


In [46]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df_weekly_price_until_max)
# df =delete_upper_path(df_weekly_price_until_max)
# display(df)

class hyperwave:
    def __init__(self, \
                 min_m = 0.5, \
                 phase2_weeks_find_max = 156, \
                 phase_grow_factor = 2, \
                 phase4_min_weeks = 15, \
                 phase4_validation_previous_high = 1.3):
        self.phase2_weeks_find_max = phase2_weeks_find_max
        self.phase_grow_factor = phase_grow_factor
        self.min_m  = min_m  
        self.phase4_min_weeks= phase4_min_weeks 
        self.phase4_validation_previous_high = phase4_validation_previous_high 

    def get_hyperwave(self, df):
        # Step 1 - Get the raw Hull from max and min raw data
        df_min_to_max = self._borne_raw_data_between_max_to_min( df )
        max_weekId = df_min_to_max.loc[ df_min_to_max.loc[:,('weekId')].argmax(), 'weekId']
        df_post_max = df.loc[max_weekId:]    
        df_hull = self._order_and_reset_index( self._delete_above_path( get_hull_tangent( df_min_to_max )))
        
        hw_phases_first_round = self._group_hyperwave_phase_1_to_4(df_hull, df)
        
        # Step 2 - Find max Price prior of start hyperwave
        first_phase_id = min(len(hw_phases_first_round), 3) * -1
        phase_2 = hw_phases_first_round [first_phase_id ]                
        min_week = self._get_phase_start_week( df_hull, phase_2 )
        max_price_weeks_before_start_week = self._get_max_price_week_before(df, min_week)
        
        hw_start_weekId = self._get_weekId_first_price_greater_than( df_min_to_max, \
                                                                    min_week, \
                                                                    max_price_weeks_before_start_week )
        
        # Step 3 - Get new Hull for the borned hyperwave raw data
        df_hyperwave_raw_data = df_min_to_max[(df_min_to_max.weekId >= hw_start_weekId )]

        df_hull_hyperwave = self._order_and_reset_index( \
                                        self._delete_above_path( get_hull_tangent( df_hyperwave_raw_data ))) 

        
        hw_phases_temp = self._group_hyperwave_phase_1_to_4(df_hull_hyperwave, df)
        
        max_nb_phases = min(len(hw_phases_temp), 3) * -1
        hw_phases_temp = hw_phases_temp[:max_nb_phases]
        print(hw_phases_temp)
        hyperwave = {}
        phase_id = 1
        for phase in hw_phases_temp:
            phase_id = phase_id + 1
            df_phase =  df_hull_hyperwave.loc[df_hull_hyperwave.loc[phase].loc[:,('ratio_error_cut')].argmin()]
            hyperwave[ phase_id ] = df_phase[self._get_columns_not_normalize(df_phase)].to_dict()
        if len(hyperwave) >= 1:
            hyperwave[1] = self._get_phase1(hyperwave[2], max_price_weeks_before_start_week)
        
        for (phase_id, phase) in hyperwave.items():
                phase["is-broken"] = self._is_price_below_line(df_post_max, phase['m'], phase['b'] )
        
        return (df_hull_hyperwave, hw_phases_temp, hyperwave )

#     def df_result_row_to_dictionary(df_result):
    
    def _is_price_below_line(self, df, m, b):
        df['phase_line_week_price'] = df["weekId"] * m + b
        return df[df["close"] < df["phase_line_week_price"]].any()
#     return df.any(axis='is_price_below') 
    
    def _get_phase1(self, dic_phase2, price_break):
        dic_phase1 = dic_phase2.copy()
        dic_phase1['angle'] = 0
        dic_phase1['b'] = price_break
        dic_phase1['index'] = 0
        dic_phase1['m'] =  0
        dic_phase1['mean_error'] = 0
        dic_phase1['nb_is_lower'] = 0
        dic_phase1['ratio_error_cut'] = 0
        dic_phase1['weeks'] = 0
        return dic_phase1

    def _get_columns_not_normalize(self, df):
        return [c for c in df.axes[0] if "normalize" not in c]

    def _group_hyperwave_phase_1_to_4(self, df_result, df_raw):
        filtered_hw = df_result[ (df_result.m_normalize > 0) ]
        
        current_phase_m = filtered_hw.iloc[0].m_normalize
        hw_phases_temp = []
        hw_current_phase = [filtered_hw.index[0]]
        
        for index, row in filtered_hw.loc[2:].iterrows():
            if row.m_normalize < current_phase_m * self.phase_grow_factor:
                hw_current_phase.append(index)
            else:
                hw_phases_temp.append(hw_current_phase)
                hw_current_phase = [index]
                current_phase_m = row.m_normalize
        
        hw_phases_temp.append(hw_current_phase)
        
#         if len(hw_phases_temp) == 3:
#             return hw_phases_temp
            
        for i in np.arange(len(hw_phases_temp)-1, 1, -1):
            phase = hw_phases_temp[i]
            current_phase_max = self._get_max_value_phase(phase, df_result, df_raw)
            previous_phase_max = self._get_max_value_phase(hw_phases_temp[i - 1], df_result, df_raw)
                                
            if self._sum_group_weeks( filtered_hw, phase) < self.phase4_min_weeks \
                or current_phase_max > previous_phase_max * self.phase4_validation_previous_high:
                hw_phases_temp.remove( phase )
                hw_phases_temp[ i - 1 ].extend( phase )

        display(hw_phases_temp)
        return hw_phases_temp

    def _get_max_phase_max(self, phase, df_result, df):
        df_phase = df_result.loc[phase]
        weekId_min = df_phase.loc[df_phase.loc[:,('x1')].argmin(), 'x1']
        weekId_max = df_phase.loc[df_phase.loc[:,('x2')].argmax(), 'x2']
        return self._get_max_value_between(df, weekId_min, weekId_max)
    
    def _get_max_value_phase(self, phase, df_result, df):
        df_phase = df_result.loc[phase]
        weekId_min = df_phase.loc[df_phase.loc[:,('x1')].argmin(), 'x1']
        weekId_max = df_phase.loc[df_phase.loc[:,('x2')].argmax(), 'x2']
        return self._get_max_value_between(df, weekId_min, weekId_max)
        
    def _get_max_value_between(self, df, weekId_min, weekId_max):
        df_phase = df.loc[weekId_min:weekId_max]
        return df_phase.loc[ df_phase.loc[:,('close')].argmax(), 'close']
        
    def _order_and_reset_index(self, df):
        return df.sort_values(['x1','x2'], ascending=[True, False]) \
            .reset_index()


    def _sum_group_weeks(self, df, group):
        return df.loc[group].sum()['weeks']
    
    def _get_weekId_first_price_greater_than(self, df, min_week_id, max_price):
        df_week_greater_than = df[(df.weekId >= min_week_id)]

        df_val_price_greater_than_max = df_week_greater_than[(df_week_greater_than.close > max_price)]
        return df_val_price_greater_than_max.loc[ df_val_price_greater_than_max.loc[:,('weekId')].argmin()]['weekId']

    def _get_phase_start_week(self, df_result, phase_lines):
        return min(df_result.iloc[phase_lines]['x1'])
        
        
    def _delete_above_path(self, df):
# As we are using Hull to find the external phase of the graph. The positive mean_error as the way up
# whereas the negative are the way down
        return df[(df.mean_error >= 0)]

        
    def _delete_below_path(self, df):
# As we are using Hull to find the external phase of the graph. The positive mean_error as the way up
# whereas the negative are the way down
        return df[(df.mean_error < 0)]

    def _borne_raw_data_between_max_to_min(self, df):
#     Born the dataframe from with all the value before weekId of Max and from them find the min to born the other side
        max_price_weekId = self._get_weekId_max_price(df)
        df_until_max = df.loc[:max_price_weekId]
        min_price_weekId = self._get_weekId_min_price( df_until_max )    
        df_min_to_max = df_until_max.loc[min_price_weekId:]
        return df_min_to_max

    def _get_weekId_max_price(self, df):
        return df.loc[ df.loc[:,('close')].argmax(), 'weekId']

    def _get_weekId_min_price(self, df):
        return df.loc[ df.loc[:,('close')].argmin(), 'weekId']

    def _get_max_price(self, df, column_name = 'close'):
        return df.loc[ df.loc[:,(column_name)].argmax()][column_name]

    def _get_max_price_week_before(self, df, weekId):
        last_n_weeks_Items = df[(df.weekId <= weekId)].tail(self.phase2_weeks_find_max)    
        max_price = self._get_max_price( last_n_weeks_Items )
        return max_price


In [47]:
hw = hyperwave()

(df_result, phases, hyperwave) = hw.get_hyperwave(df_weekly_price)
# display(df_result)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_result)
    display(phases)
    display(hyperwave)
# display(df_result.loc[2:])
# hw._get_hw_start_week(df_result)

[[0], [2, 3, 4, 5, 6, 7]]

/home/nbuser/anaconda3_501/lib/python3.6/site-packages/ipykernel/__main__.py:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



[[4, 4, 5], [6, 7, 8, 9, 10, 11, 12]]

[]


,index,x1,x1_date,x1_normalize,x2,x2_date,x2_normalize,y1,y1_normalize,y2,y2_normalize,m,b,m_normalize,b_normalize,angle,angle_normalize,weeks,mean_error,nb_is_lower,ratio_error_cut,ratio_slope_y1_normalize,ratio_slope_y2_normalize
0,3,10284,1997-02-10,0.000000,10288,1997-03-10,0.003670,2.65,0.013184,2.38,0.007791,-0.067500,696.820000,-1.469736,0.013184,-3.861607,-55.768858,4,49.464979,3,16.488326,-0.008970,-0.005301
1,4,10288,1997-03-10,0.003670,10290,1997-03-24,0.005505,2.38,0.007791,2.28,0.005793,-0.050000,516.780000,-1.088694,0.011786,-2.862405,-47.431528,2,39.997479,3,13.332493,-0.007156,-0.005321
2,6,10290,1997-03-24,0.005505,10296,1997-05-05,0.011009,2.28,0.005793,2.01,0.000400,-0.045000,465.330000,-0.979824,0.011187,-2.576572,-44.416136,6,37.302479,4,9.325620,-0.005912,-0.000408
3,5,10296,1997-05-05,0.011009,10298,1997-05-19,0.012844,2.01,0.000400,1.99,0.000000,-0.010000,104.970000,-0.217739,0.002797,-0.572939,-12.283779,2,18.647479,3,6.215826,-0.001835,-0.000000
4,9,10298,1997-05-19,0.012844,10424,1999-10-18,0.128440,1.99,0.000000,2.34,0.006992,0.002778,-26.615556,0.060483,-0.000777,0.159155,3.461203,126,11.862479,5,2.372496,0.000000,0.115596
5,10,10424,1999-10-18,0.128440,10898,2008-11-17,0.563303,2.34,0.006992,3.87,0.037555,0.003228,-31.307089,0.070283,-0.002036,0.184941,4.020294,474,11.680201,3,3.893400,0.099478,0.534341
6,2,10898,2008-11-17,0.563303,11047,2011-09-26,0.700000,3.87,0.037555,8.43,0.128646,0.030604,-329.652685,0.666368,-0.337812,1.752934,33.678225,149,13.569157,4,3.392289,0.056358,0.193055
7,7,11047,2011-09-26,0.700000,11275,2016-02-08,0.909174,8.43,0.128646,23.52,0.430084,0.066184,-722.706974,1.441086,-0.880115,3.786554,55.242412,228,21.325637,4,5.331409,0.089270,0.298444
8,8,11275,2016-02-08,0.909174,11321,2016-12-26,0.951376,23.52,0.430084,26.78,0.495206,0.070870,-775.534348,1.543105,-0.972867,4.053749,57.054982,46,23.415305,2,11.707653,0.278713,0.320915
9,13,11321,2016-12-26,0.951376,11357,2017-09-04,0.984404,26.78,0.495206,35.85,0.676388,0.251944,-2825.483056,5.485806,-4.723859,14.141050,79.669064,36,112.504146,7,16.072021,0.090270,0.123298


[]

{}

In [23]:
df = df_weekly_price.loc[:11055]
display(df.index)
display(df)

# df.index.idxmax()

Int64Index([10224, 10225, 10226, 10227, 10228, 10229, 10230, 10231, 10232,
            10233,
            ...
            11046, 11047, 11048, 11049, 11050, 11051, 11052, 11053, 11054,
            11055],
           dtype='int64', name='weekId', length=832)

,Date,open,high,low,close,date,is_price_closing_up,weekId
weekId,,,,,,,,
10224,"Dec 22, 1995",2.43,2.49,2.41,2.43,1995-12-22,False,10224
10225,"Dec 29, 1995",2.38,2.49,2.38,2.49,1995-12-29,False,10225
10226,"Jan 05, 1996",2.54,2.54,2.49,2.49,1996-01-05,True,10226
10227,"Jan 12, 1996",2.36,2.38,2.33,2.33,1996-01-12,True,10227
10228,"Jan 19, 1996",2.28,2.28,2.17,2.20,1996-01-19,False,10228
10229,"Jan 26, 1996",2.36,2.46,2.33,2.33,1996-01-26,False,10229
10230,"Feb 02, 1996",2.43,2.46,2.43,2.43,1996-02-02,False,10230
10231,"Feb 09, 1996",2.54,2.65,2.54,2.59,1996-02-09,False,10231
10232,"Feb 16, 1996",2.62,2.62,2.59,2.59,1996-02-16,True,10232


In [24]:
df = df_weekly_price_until_max
max_weekId = 9689
nb_tail_items = 156

def get_max_price(df, column_name = 'close'):
    return df.loc[ df.loc[:,(column_name)].argmax()][column_name]
# weekId_max_price = get_weekId_max_price(last_n_weeks_Items) 
#     max_price_last_n_weeks = last_n_weeks_Items[(last_n_weeks_Items.weekId == weekId_max_price )].iloc[0]

last_n_weeks_Items = df[(df.weekId <= max_weekId)].tail(nb_tail_items)    
max_price = get_max_price(last_n_weeks_Items )

print( max_price)

df_week_greater_than = df[(df.weekId >= max_weekId)]

# df.loc[ df.loc[:,('close')].argmax()]['weekId']
df_val_price_greater_than_max = df_week_greater_than[(df_week_greater_than.close > max_price)]
weekId_start_hyperwave = df_val_price_greater_than_max.loc[ df_val_price_greater_than_max.loc[:,('weekId')].argmin()]['weekId']
df_week_greater_than = df[(df.weekId >= weekId_start_hyperwave )]

df = get_normalize_column(df_week_greater_than, 'close')
df = get_normalize_column(df, 'weekId')

df_hyperwave = get_hull_tangent(df,"weekId_normalize", "close_normalize").sort_values(['x1','y2'], ascending=[True, False])


df_hyperwave = delete_shorter_path(df_hyperwave)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df_hyperwave)
py.iplot(graph_hyperwave(df_weekly_price_until_max,df_hyperwave), filename='hyperwave')
# py.iplot(fig, filename='shapes-lines')

ValueError: attempt to get argmax of an empty sequence

In [ ]:

# display(df_hyperwave.sort_values(['x1','y2'], ascending=[True, False]))
# print(max_price_weekId)
# print(get_upper_path(df_hyperwave))
df_low_path = delete_upper_path(df_hull_tangent)
display(df_hull_tangent)
display(df_low_path )
print(df_low_path.std()) 
print(df_low_path.mean()) #(axis=['m'])
print(df_low_path.median())

In [ ]:
# def get_line_slope(row):
#     return (row.close_from - row.close_to) / (row.weekId_from - row.weekId_to)


# weekId_close_from_to['slope'] = weekId_close_from_to.apply (lambda row: get_line_slope(row),axis=1)



# weekId_close_from_to

# # m = -0.110000
# # b = 1092.930000
# # m = 15.82
# # b = -177675.62
# def is_phase_below(m, b):
# #     m = row.m
# #     b = row.b
#     df = df_weekly_price
#     df_weekly_close = df[["weekId", "close"]]
#     df_weekly_close['ln_y'] = (df_weekly_close.weekId * m) + b
#     df_weekly_close['is_below'] = df_weekly_close.ln_y.le(df_weekly_close.close) | np.isclose(df_weekly_close['ln_y'], df_weekly_close['close'])
#     return df_weekly_close['is_below'].all()


# weekId_close_from_to['is_below'] = weekId_close_from_to.apply (lambda row: is_phase_below(row, df_weekly_price),axis=1)
# is_phase_below()

# weekId_close_from_to['r'] = weekId_close_from_to(weekId_close_from_to['m'], weekId_close_from_to['b'] )

# weekId_close_from_to
# import numpy as np
# import pandas as pd
# np.seterr(divide='ignore', invalid='ignore')



# df = df_weekly_price_until_max[:15].rename(columns={'weekId':'x', 'close': 'y'})


# data = {'x':[1,2,3,4],'y':[5,6,10,12]}
# df = pd.DataFrame(data)



# calculate m and b from y = mx = b
# df_m = (df['y'].values - df['y'].values[:, None]) / (df['x'].values - df['x'].values[:, None])
# df_b = df['y'].values - (df['x'].values * df_m)


# import itertools
# nb_partition = int((df['x'].values.shape[0] / 10)) + 1
# x_values_split = np.array_split(df['x'].values, nb_partition) 
# y_values_split = np.array_split(df['y'].values, nb_partition)              


# def get_y_values(x_value, m, b):
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
#     x_cube = np.reshape(x_values, cube_shape)
    
#     return (x_cube * m) + b 

# def get_is_line_above_close(x_values, y_values, m, b):
#     y_lenght = y_values.shape[0]
#     cube_shape = (y_lenght, 1, 1)
    
#     y_cube = np.reshape(y_values, cube_shape)
#     return get_y_values(x_values,m, b) < y_cube

    
# result = [get_y_vlowalues(x_values, df_m, df_b) for (x_values, y_values) in itertools.zip_longest(x_values_split, y_values_split )]
#     x_lenght = x_values.shape[0]
#     cube_shape = (x_lenght, 1, 1)
    
#     x_cube = np.reshape(x_values, cube_shape)
#     y_cube = np.reshape(y_values, cube_shape)
#     cube_y_value_calculated =  (x_cube * df_m) + df_b 

# x_values = x_values_split[0]
# y_values = y_values_split[0]              
# x_lenght = x_values.shape[0]
# cube_shape = (x_lenght, 1, 1)
https://www.google.com/search?q=hindsight+meaning&ie=utf-8&oe=utf-8&client=firefox-b-ab
# x_cube = np.reshape(x_values, cube_shape)
# y_cube = np.reshape(y_values, cube_shape)
